In [ ]:

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets
from torchvision import transforms

In [ ]:
transf = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Load the training set
train_dataset = datasets.STL10(root="/Dataset", split='train', download=True, transform=transf)

# Load the test set
test_dataset = datasets.STL10(root="/Dataset", split='test', download=False, transform=transf)

In [ ]:
##Specification of batch size
BATCH_SIZE =64
## Split 20% of train dataset for validation (to use in early stopping)
VALID_SPLIT = 0.2
###definition of seed number for cnn to generate same results every time
SEED = 42
np.random.seed(SEED)
##Specification of learning rate( Trial and error)
LEARNING_RATE = 0.001
### Determination of epochs (we specify high number and this will be stopped by early stopping)
N_EPOCHS = 100
##Spliting validation and train by shuffling their indices to make it random
n_train = len(train_dataset)
indices = list(range(n_train))
np.random.shuffle(indices)
split = int(np.floor(VALID_SPLIT * n_train))
train_indices, valid_indices = indices[split:], indices[:split]

In [ ]:
##Sample data randomly by use of subset random sampler
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)
##creating data loaders to avoid memory limit issues
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           sampler=valid_sampler)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=BATCH_SIZE)

In [ ]:

# Check if GPU is available
gpu_availabe = torch.cuda.is_available()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input: 64x64x3
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Output: 64x64x16
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # Output: 32x32x32
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)  # Output: 16x16x64
        
        self.pool = nn.MaxPool2d(2, 2) # Output: 16/2x16/2x64

        # Fully connected layers
        self.fc1 = nn.Linear(8 * 8 * 64, 512)  # to match the new flattened size
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        
        # Dropout rate
        self.dropout = nn.Dropout(0.15)

    def forward(self, x):
        # Convolutional layers + pooling
        x = self.pool(F.relu(self.conv1(x)))  # 64x64 -> 32x32
        x = self.pool(F.relu(self.conv2(x)))  # 32x32 -> 16x16
        x = self.pool(F.relu(self.conv3(x)))  # 16x16 -> 8x8

        # Flatten the tensor
        x = x.view(-1, 8 * 8 * 64)  # Flattened size: 4096

        # Fully connected layers + dropout
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

# load the model
model = Net()
print(model)

# Move to GPU if available
if gpu_availabe:
    model.cuda()


Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.15, inplace=False)
)


In [ ]:
# specifing loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Early stopping parameters 
##How much should it wait?
patience = 10
##How much the minimum improvement should be?
min_delta = 0.001
early_stop = False
##To track the numbers improvements weren't made
counter = 0
best_accuracy = 0.0  # to track the best accuracy

for epoch in range(1, N_EPOCHS + 1):

    print(f"Epoch {epoch}")

    # Keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    ##For usage in accuracy
    valid_correct = 0
    valid_total = 0

    # Train the model 

    model.train()
    for data, target in train_loader:
        # Move tensors to GPU if CUDA is available
        if gpu_availabe:
            data, target = data.cuda(), target.cuda()

        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        loss = criterion(output, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update training loss
        train_loss += loss.item() * data.size(0)
  
    # Validate the model 

    model.eval()
    with torch.no_grad():
        for data, target in valid_loader:
            if gpu_availabe:
                data, target = data.cuda(), target.cuda()

            output = model(data)
            loss = criterion(output, target)

            # Update validation loss
            valid_loss += loss.item() * data.size(0)

            # Calculate validation accuracy
            _, predicted = torch.max(output, 1)
            valid_correct += (predicted == target).sum().item()
            valid_total += target.size(0)

    # Calculate average losses
    train_loss = train_loss / len(train_indices)
    valid_loss = valid_loss / len(valid_indices)

    # Calculate validation accuracy
    valid_accuracy = 100 * valid_correct / valid_total

    # Print training and validation statistics 
    print(f"Epoch: {epoch}\tTraining Loss: {train_loss:.6f}\tValidation Accuracy: {valid_accuracy:.2f}%")

    # Check for early stopping based on accuracy
    if valid_accuracy > best_accuracy + min_delta:
        print(f"Validation accuracy increased ({best_accuracy:.2f}% --< {valid_accuracy:.2f}%). Saving model...")
        torch.save(model.state_dict(), "model_cnn.h5")
        best_accuracy = valid_accuracy
        counter = 0  # Reset counter when improvement is observed
    else:
        counter += 1
        print(f" Early stopping counter: {counter}/{patience}")

    # Check if early stopping should be triggered
    if counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}. Best validation accuracy: {best_accuracy:.2f}%")
        break


Epoch 1
Epoch: 1	Training Loss: 2.283858	Validation Accuracy: 20.70%
Validation accuracy increased (0.00% --< 20.70%). Saving model...
Epoch 2
Epoch: 2	Training Loss: 2.073677	Validation Accuracy: 26.80%
Validation accuracy increased (20.70% --< 26.80%). Saving model...
Epoch 3
Epoch: 3	Training Loss: 1.905711	Validation Accuracy: 31.60%
Validation accuracy increased (26.80% --< 31.60%). Saving model...
Epoch 4
Epoch: 4	Training Loss: 1.827428	Validation Accuracy: 32.70%
Validation accuracy increased (31.60% --< 32.70%). Saving model...
Epoch 5
Epoch: 5	Training Loss: 1.766169	Validation Accuracy: 34.30%
Validation accuracy increased (32.70% --< 34.30%). Saving model...
Epoch 6
Epoch: 6	Training Loss: 1.724817	Validation Accuracy: 35.20%
Validation accuracy increased (34.30% --< 35.20%). Saving model...
Epoch 7
Epoch: 7	Training Loss: 1.684025	Validation Accuracy: 37.30%
Validation accuracy increased (35.20% --< 37.30%). Saving model...
Epoch 8
Epoch: 8	Training Loss: 1.643750	Validati

In [ ]:
# load best model
model.load_state_dict(torch.load("model_cnn.h5"))

# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()  # evaluation mode 

# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if gpu_availabe:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs 
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not gpu_availabe else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each class
    for i in range(BATCH_SIZE):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# averaging test loss
test_loss = test_loss/len(test_loader.dataset)
print(f"Test Loss: {test_loss:.6f}")

classes = test_dataset.classes

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.307749
Test Accuracy of airplane: 75% (603/800)
Test Accuracy of  bird: 41% (331/800)
Test Accuracy of   car: 68% (547/800)
Test Accuracy of   cat: 41% (333/800)
Test Accuracy of  deer: 41% (334/800)
Test Accuracy of   dog: 41% (333/800)
Test Accuracy of horse: 68% (549/800)
Test Accuracy of monkey: 40% (322/800)
Test Accuracy of  ship: 68% (544/800)
Test Accuracy of truck: 65% (522/800)

Test Accuracy (Overall): 55% (4418/8000)


In [ ]:
#Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
import optuna

train_on_gpu = torch.cuda.is_available()

class Net(nn.Module):
    def __init__(self, num_filters1=16, num_filters2=32, num_filters3=64, fc1_size=512, dropout_rate=0.2):
        super(Net, self).__init__()
        # 64x64x3
        self.conv1 = nn.Conv2d(3, num_filters1, 3, padding=1)
        # 32x32xnum_filters1
        self.conv2 = nn.Conv2d(num_filters1, num_filters2, 3, padding=1)
        # 16x16xnum_filters2
        self.conv3 = nn.Conv2d(num_filters2, num_filters3, 3, padding=1)
        # 8x8xnum_filters3
        
        self.pool = nn.MaxPool2d(2, 2)

        # Adjusted flattened size to 8*8*num_filters3
        self.fc1 = nn.Linear(8*8*num_filters3, fc1_size)
        self.fc2 = nn.Linear(fc1_size, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, x.size(1) * 8 * 8)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to be optimized
    num_filters1 = trial.suggest_int("num_filters1", 16, 64, step=16)
    num_filters2 = trial.suggest_int("num_filters2", 32, 128, step=16)
    num_filters3 = trial.suggest_int("num_filters3", 64, 256, step=32)
    fc1_size = trial.suggest_int("fc1_size", 256, 1024, step=128)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)

    # Create model
    model = Net(num_filters1, num_filters2, num_filters3, fc1_size, dropout_rate)
    if train_on_gpu:
        model.cuda()

    # Define optimizer and loss function
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Training loop (single epoch for demonstration)
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)

    train_loss /= len(train_loader.dataset)
    return train_loss

# Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

print("Best hyperparameters:", study.best_params)


[I 2025-05-08 21:27:14,532] A new study created in memory with name: no-name-c4251abd-ae66-4f77-9b08-b67cffd49769
[I 2025-05-08 21:27:19,308] Trial 0 finished with value: 3.1067285079956055 and parameters: {'num_filters1': 64, 'num_filters2': 48, 'num_filters3': 160, 'fc1_size': 1024, 'dropout_rate': 0.30000000000000004}. Best is trial 0 with value: 3.1067285079956055.
[I 2025-05-08 21:27:23,639] Trial 1 finished with value: 2.6221385948181153 and parameters: {'num_filters1': 64, 'num_filters2': 48, 'num_filters3': 224, 'fc1_size': 512, 'dropout_rate': 0.30000000000000004}. Best is trial 1 with value: 2.6221385948181153.
[I 2025-05-08 21:27:27,653] Trial 2 finished with value: 2.1947441864013673 and parameters: {'num_filters1': 48, 'num_filters2': 48, 'num_filters3': 160, 'fc1_size': 768, 'dropout_rate': 0.5}. Best is trial 2 with value: 2.1947441864013673.
[I 2025-05-08 21:27:32,866] Trial 3 finished with value: 2.1916861923217774 and parameters: {'num_filters1': 48, 'num_filters2': 1

Best hyperparameters: {'num_filters1': 16, 'num_filters2': 64, 'num_filters3': 64, 'fc1_size': 384, 'dropout_rate': 0.1}
